- 使用tushare包获取某股票的历史行情数据
- 输出该股票所有收盘比开盘上涨3%以上的日期
- 输出该股票所有开盘比前日收盘跌幅超过2%的日期
- 假如我从2010年1月1日开始，每月第一个交易日买入1手股票，每年最后一个交易日卖出所有股票，到今天为止，我的收益如何?

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

import tushare as ts

In [3]:
# 使用tushare包获取某股票的历史行情数据
df = ts.get_k_data(code='600519', start='2001')
df

,date,open,close,high,low,volume,code
0,2001-08-27,5.392,5.554,5.902,5.132,406318.00,600519
1,2001-08-28,5.467,5.759,5.781,5.407,129647.79,600519
2,2001-08-29,5.777,5.684,5.781,5.640,53252.75,600519
3,2001-08-30,5.668,5.796,5.860,5.624,48013.06,600519
4,2001-08-31,5.804,5.782,5.877,5.749,23231.48,600519
...,...,...,...,...,...,...,...
4321,2019-10-14,1186.000,1180.000,1187.660,1170.000,27187.00,600519
4322,2019-10-15,1180.000,1211.000,1215.680,1177.030,40034.00,600519
4323,2019-10-16,1168.000,1170.000,1187.500,1160.000,59150.00,600519
4324,2019-10-17,1176.000,1183.330,1186.660,1162.500,30508.00,600519


In [4]:
df.to_csv('./maotai.csv')

In [6]:
# 将date列的数据转成时间序列，然后将该列作为整个数据源的行索引
data = pd.read_csv('./maotai.csv', index_col='date', parse_dates=['date'])
data.drop(labels="Unnamed: 0", axis=1, inplace=True)  # inplace=True表示将修改后的数据作用回原表
data

,open,close,high,low,volume,code
date,,,,,,
2001-08-27,5.392,5.554,5.902,5.132,406318.00,600519
2001-08-28,5.467,5.759,5.781,5.407,129647.79,600519
2001-08-29,5.777,5.684,5.781,5.640,53252.75,600519
2001-08-30,5.668,5.796,5.860,5.624,48013.06,600519
2001-08-31,5.804,5.782,5.877,5.749,23231.48,600519
...,...,...,...,...,...,...
2019-10-14,1186.000,1180.000,1187.660,1170.000,27187.00,600519
2019-10-15,1180.000,1211.000,1215.680,1177.030,40034.00,600519
2019-10-16,1168.000,1170.000,1187.500,1160.000,59150.00,600519


In [7]:
data.index[0]

Timestamp('2001-08-27 00:00:00')

In [10]:
(data['close'] -  data['open'])/data['open'] > 0.03

date
2001-08-27     True
2001-08-28     True
2001-08-29    False
2001-08-30    False
2001-08-31    False
              ...  
2019-10-14    False
2019-10-15    False
2019-10-16    False
2019-10-17    False
2019-10-18    False
Length: 4326, dtype: bool

In [12]:
# 输出该股票所有收盘比开盘上涨3%以上的日期
# (收盘价-开盘价)/开盘价 > 0.03
indexs = (data['close'] -  data['open'])/data['open'] > 0.03
data.loc[indexs].index

DatetimeIndex(['2001-08-27', '2001-08-28', '2001-09-10', '2001-12-21',
               '2002-01-18', '2002-01-31', '2003-01-14', '2003-10-29',
               '2004-01-05', '2004-01-14',
               ...
               '2019-03-01', '2019-03-18', '2019-04-10', '2019-04-16',
               '2019-05-10', '2019-05-15', '2019-06-11', '2019-06-20',
               '2019-09-12', '2019-09-18'],
              dtype='datetime64[ns]', name='date', length=303, freq=None)

In [13]:
data['close']

date
2001-08-27       5.554
2001-08-28       5.759
2001-08-29       5.684
2001-08-30       5.796
2001-08-31       5.782
                ...   
2019-10-14    1180.000
2019-10-15    1211.000
2019-10-16    1170.000
2019-10-17    1183.330
2019-10-18    1175.790
Name: close, Length: 4326, dtype: float64

In [14]:
data['close'].shift(1)

date
2001-08-27         NaN
2001-08-28       5.554
2001-08-29       5.759
2001-08-30       5.684
2001-08-31       5.796
                ...   
2019-10-14    1174.600
2019-10-15    1180.000
2019-10-16    1211.000
2019-10-17    1170.000
2019-10-18    1183.330
Name: close, Length: 4326, dtype: float64

In [17]:
# 输出该股票所有开盘比前日收盘跌幅超过2%的日期
indexs = (data['open'] - data['close'].shift(1)) / data['close'].shift(1) < -0.02
data.loc[indexs].index

DatetimeIndex(['2001-09-12', '2002-06-26', '2002-12-13', '2004-07-01',
               '2004-10-29', '2006-08-21', '2006-08-23', '2007-01-25',
               '2007-02-01', '2007-02-06', '2007-03-19', '2007-05-21',
               '2007-05-30', '2007-06-05', '2007-07-27', '2007-09-05',
               '2007-09-10', '2008-03-13', '2008-03-17', '2008-03-25',
               '2008-03-27', '2008-04-22', '2008-04-23', '2008-04-29',
               '2008-05-13', '2008-06-10', '2008-06-13', '2008-06-24',
               '2008-06-27', '2008-08-11', '2008-08-19', '2008-09-23',
               '2008-10-10', '2008-10-15', '2008-10-16', '2008-10-20',
               '2008-10-23', '2008-10-27', '2008-11-06', '2008-11-12',
               '2008-11-20', '2008-11-21', '2008-12-02', '2009-02-27',
               '2009-03-25', '2009-08-13', '2010-04-26', '2010-04-30',
               '2011-08-05', '2012-03-27', '2012-08-10', '2012-11-22',
               '2012-12-04', '2012-12-24', '2013-01-16', '2013-01-25',
      

In [9]:
# 假如我从2010年1月1日开始，每月第一个交易日买入1手股票，每年最后一个交易日卖出所有股票，到今天为止，我的收益如何?

In [4]:
# 找出每年的第一个交易日对应的行数据(数据的重新取样)

In [35]:
df = ts.get_k_data(code='600159', start='2010', end='2019')
df.to_csv('贵州茅台.csv')

In [41]:
data = pd.read_csv('./贵州茅台.csv', index_col='date', parse_dates=['date']).drop(labels="Unnamed: 0", axis=1)
data

,open,close,high,low,volume,code
date,,,,,,
2010-01-04,8.776,9.014,9.227,8.776,103212.07,600159
2010-01-05,9.004,8.946,9.116,8.732,62011.59,600159
2010-01-06,8.917,9.305,9.576,8.800,126523.53,600159
2010-01-07,9.145,9.188,9.402,9.019,91887.24,600159
2010-01-08,9.159,9.285,9.343,8.960,58919.87,600159
2010-01-11,9.295,8.912,9.358,8.728,80804.62,600159
2010-01-12,8.868,9.213,9.334,8.796,74142.36,600159
2010-01-13,9.009,8.732,9.009,8.582,86259.30,600159
2010-01-14,8.655,8.985,9.048,8.655,51874.23,600159


In [23]:
# 使用tushare包获取某股票的历史行情数据
df = ts.get_k_data(code='600519', start='2010-01-01')
df.to_csv('./贵州茅台.csv')

# 读取文件数据
data = pd.read_csv('./贵州茅台.csv', index_col='date', parse_dates=['date'])
data.drop(labels='Unnamed: 0', axis=1, inplace=True)

price_last = data['open'][-1]  # 2019年最后一个交易日的开盘价
df = data['2010':'2019']  # 剔除首尾无用的数据

#　Pandas提供了resample函数用便捷的方式对时间序列进行重彩样,根据时间粒度的变大或者变小分为降采样和升采样
df_monthly = df.resample('M').first()
df_yearly = df.resample('Y').last()[:-1]  # 去除最后一年

cost_money = 0  # 基础资金
hold = 0  # 每年持有的股票
for year in range(2010, 2020):
    cost_money -= df_monthly.loc[str(year)]['open'].sum() * 100
    hold += len(df_monthly[str(year)]["open"]) * 100

    if year != 2019:
        cost_money += df_yearly.loc[str(year)]["open"][0] * hold
        hold = 0  # 每年持有的股票

cost_money += hold * price_last

print(cost_money)

558598.7
